# **Image Super-Resolution**
**64x64 -> 512x512**

A colab notebook for upscaling 64x64 images to 512x512, using [this](https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement).


## Setup
---
Instructions:

*   Turn on hardware acceleration under `Runtime -> Change Runtime Type -> Hardware accelerator -> GPU`
*   Use this command to ensure that the connected machine has a GPU:

In [4]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla V100-PCIE-32GB, 535.154.05, 32768 MiB


*    Now, execute each cell sequentially, waiting until each one is done before running the next cell.


### Clone repo, download a pre-trained model, install dependencies.


In [2]:
%cd /content/
!git clone https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement.git

[Errno 2] No such file or directory: '/content/'
/scratch/rv1057
Cloning into 'Image-Super-Resolution-via-Iterative-Refinement'...
remote: Enumerating objects: 569, done.
remote: Counting objects: 100% (568/568), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 569 (delta 298), reused 472 (delta 267), pack-reused 1 (from 1)
Receiving objects: 100% (569/569), 10.01 MiB | 56.60 MiB/s, done.
Resolving deltas: 100% (298/298), done.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.2 MB/s eta 0:00:00


In [15]:
!pip install tensorboardx
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 103.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 19.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 28.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 136.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 23.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 37.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install lmdb
!pip install gdown

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**NOTE: The `gdown` command does not always work.** If it gives you an error, as a temporary workaround, you must manually upload files for the pre-trained model. Download the files [here](https://drive.google.com/drive/folders/1mCiWhFqHyjt5zE4IdA41fjFwCYdqDzSF) (you may have to log into your google account to be able to do so). Then, click the `files` button on the left hand side of the screen, and find the folder labled `Image-Super-Resolution-via-Iterative-Refinement`. Click the three dots on the right of the folder, and click `upload`. Now, select the model files that you downloaded.

**ALTERNATIVELY: if you plan to use this script frequently, you can make a copy of the model files in your own google drive.** Go [here](https://drive.google.com/drive/folders/1mCiWhFqHyjt5zE4IdA41fjFwCYdqDzSF), right click on each file, and hit `Make a copy`. Next, locate the files in your own google drive, and rename them from `Copy of I830000_E32_opt.pth` and `Copy of I830000_E32_gen.pth` to `I830000_E32_opt.pth` and `I830000_E32_gen.pth`. Next, right click on each file, and hit `Get link`. Click on `Restricted`, and change it to `Anyone with the link...`, as `Viewer`. Now, copy the links. They should look something like this: `https://drive.google.com/file/d/<file id>/view?usp=sharing` - copy the ids, and paste them into the `gdown` commands below.  

For some reason, the model files are getting rate limited at times. This will hopefully be fixed in the future.

In [6]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!gdown --id 1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr
!gdown --id 1Vwe1Raa-Dpop75o5v7O5qvGoOdFDlyc-
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/config/

[Errno 2] No such file or directory: '/content/Image-Super-Resolution-via-Iterative-Refinement/'
/scratch/rv1057
/home/rv1057/.local/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr
From (redirected): https://drive.google.com/uc?id=1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr&confirm=t&uuid=a398f2e5-ff55-4930-b0f9-86acf891e6a3
To: /scratch/rv1057/I830000_E32_opt.pth
100%|██████████████████████████████████████| 1.24G/1.24G [00:25<00:00, 48.0MB/s]
/home/rv1057/.local/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Vwe1Raa-D

Once you have done training at least once and saved it somewhere (preferably on Google Drive), you can use your trained checkpoint instead of the original model. Like for checkpoints saved in https://drive.google.com/drive/u/0/folders/1YPr5TWcHiDhifOSqS_ZaD36Zm7WDZdGR, you can just get them using the procedure from above: **Alternatively**

In [7]:
%cd /Image-Super-Resolution-via-Iterative-Refinement/
!gdown --id 1-HpBSYQs_QrUDrdIXo1jEQ8JZf17pO8z
!gdown --id 1-ADxQK2BKiUtjocpWAoqzIDpTYW9Qwxi
%cd /Image-Super-Resolution-via-Iterative-Refinement/config/

[Errno 2] No such file or directory: '/Image-Super-Resolution-via-Iterative-Refinement/'
/scratch/rv1057
/home/rv1057/.local/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-HpBSYQs_QrUDrdIXo1jEQ8JZf17pO8z
From (redirected): https://drive.google.com/uc?id=1-HpBSYQs_QrUDrdIXo1jEQ8JZf17pO8z&confirm=t&uuid=0cca3b1a-9bce-4b9b-884f-36798bc798ff
To: /scratch/rv1057/I831000_E532_opt.pth
100%|██████████████████████████████████████| 1.24G/1.24G [00:26<00:00, 47.3MB/s]
/home/rv1057/.local/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-ADxQK2BKiUtjocp

### Patch config files

Create patchfile



In [8]:
%%writefile 64_512.patch
--- sr_sr3_64_512_new.json	2021-10-22 16:20:20.901133618 +0000
+++ sr_sr3_64_512.json	2021-10-22 16:20:52.036081672 +0000
@@ -9,8 +9,8 @@
         "tb_logger": "tb_logger",
         "results": "results",
         "checkpoint": "checkpoint",
-        "resume_state": null
-        // "resume_state": "experiments/distributed_high_sr_ffhq_210901_121212/checkpoint/I830000_E32" //pretrain model or training state
+        // "resume_state": null
+        "resume_state": "I830000_E32" //pretrain model or training state
     },
     "datasets": {
         "train": {



Writing 64_512.patch


Apply patchfile

In [13]:
#!apt-get install dos2unix
!dos2unix Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json
!patch < 64_512.patch

dos2unix: converting file Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json to Unix format...
can't find file to patch at input line 3
Perhaps you should have used the -p or --strip option?
The text leading up to this was:
--------------------------
|--- sr_sr3_64_512_new.json	2021-10-22 16:20:20.901133618 +0000
|+++ sr_sr3_64_512.json	2021-10-22 16:20:52.036081672 +0000
--------------------------
File to patch: ^C


### Prepair Data
Upload 64x64 pixel image(s) to be upscaled

*Click the **browse** button and select the images you would like to upscale.*



Create a folder for a dataset. Folder structure:

*   lr_64 (64 by 64) low-res images
*   hr_512 (512 by 512) high-res images, ground-truth. If you do not have it, the system will put a copy of sr_64_512 with name hr_512 when you call prepare_data.py
*   sr_64_512 (512 by 512) basic upscaled (bicubic or something) versions of low-res images from lr_64. This will be used as a starting image for inference/training. You DO NOT need to create it, call prepare_data.py to generate it from lr_64.

Corresponding images in all 3 datasets should have the same names.



Move your dataset to dataset folder, make sure it follows the instructions from above. Update the sr_sr3_64_512.json with your datafile. Alternatively, just replace appropriate folders in celebahq_64_512 and ffhq_64_512 with yours, delete sr_64_512.

Generate neccesary files and directory structure to begin upscaling.  

In [2]:
%ls

64_512.patch          Image-Super-Resolution-via-Iterative-Refinement/
I830000_E32_gen.pth   Running_and_Training_SR3_HPC.ipynb
I830000_E32_opt.pth   big_data/
I831000_E532_gen.pth  content/
I831000_E532_opt.pth


In [7]:
%cd Image-Super-Resolution-via-Iterative-Refinement/dataset
!gdown --id --folder 1CzPGcF4azeZ1GsUpIZasXIA6Exh_4gyy

[Errno 2] No such file or directory: 'Image-Super-Resolution-via-Iterative-Refinement/dataset'
/scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset
/home/rv1057/.local/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Retrieving folder 1h3qDSB_4Sdxk-JXZ9BGiA2XksVwIqFkQ hr_512
Processing file 1lDyOBf5PB-Q3kYIprwpUz_Z23EZ1aj7z ER.png
Processing file 1K2Mzp8rIBEZTvy_3bLx8s6sXxVS5tGUa f_non.png
Processing file 1Rj-EHbrlL_MvbUiYqj2aSAw6Qf3ifw8_ micro.png
Retrieving folder 1a2DMHmRJNAv_2CgOnxOhbftrD-AdOJc2 lr_64
Processing file 1zQvRz9p-OXA3CbXg5mgnHbFj9M7g3luJ ER.png
Processing file 11zMwCj7dwAaTlQANIAo_vFrafHpElcVr f_non.png
Processing file 1yb2tvoNaYTJK4HIAm628QgEmnjj1fHA3 micro.png
Retrieving folder contents completed
Building directory structure
Building directory structure completed


Call prepare_data.py on each dataset

In [21]:
%ls

64_512.patch          Image-Super-Resolution-via-Iterative-Refinement/
I830000_E32_gen.pth   Running_and_Training_SR3_HPC.ipynb
I830000_E32_opt.pth   big_data/
I831000_E532_gen.pth  content/
I831000_E532_opt.pth


In [1]:
%cp /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/Microtubules/high_res/Cell_001_RawSIMData_gt.png /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/Microtubules_val/high_res/

In [25]:
!python /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/data/prepare_data.py --path /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/F-actin_Nonlinear/a/low_res --size 64,512 --out /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/train_FNa

498/500 images processed                                                                                                                                                                                                                                                                                                                                                                           

In [26]:
!python /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/data/prepare_data.py --path /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/F-actin_Nonlinear_val/a/low_res --size 64,512 --out /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/val_FNa

0/2 images processed  

In [27]:
%rm -r /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/train_FNa_64_512/hr_512

In [28]:
%rm -r /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/val_FNa_64_512/hr_512

In [29]:
%cp -r /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/F-actin_Nonlinear/a/high_res /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/train_FNa_64_512

In [30]:
%cp -r /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/F-actin_Nonlinear/a/high_res /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/val_FNa_64_512

In [23]:
%ls /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/dataset/train_64_512/sr_64_512 | wc -l

3121


## Upscaling
---
For inference run this code, it takes ~10 minutes on T4. This will generate a folder on experiments (every job gets a new folder). In the folder, results contains your resulting iamges. The codes are the same as for datasets: inf means upscaled low-res image, hr means ground-truth, sr means the output of diffusion model.

In [28]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 60.3 MB/s eta 0:00:00:00:0100:01


In [1]:
%cd Image-Super-Resolution-via-Iterative-Refinement

/scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement


In [37]:
!python /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/infer.py -c /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json

export CUDA_VISIBLE_DEVICES=0,1,2
24-12-20 20:56:15.306 - INFO:   name: distributed_high_sr_ffhq
  phase: val
  gpu_ids: [0, 1, 2]
  path:[
    log: experiments/distributed_high_sr_ffhq_241220_205615/logs
    tb_logger: experiments/distributed_high_sr_ffhq_241220_205615/tb_logger
    results: experiments/distributed_high_sr_ffhq_241220_205615/results
    checkpoint: experiments/distributed_high_sr_ffhq_241220_205615/checkpoint
    resume_state: /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/experiments/distributed_high_sr_ffhq_241219_102240/checkpoint/I890000_E251
    experiments_root: experiments/distributed_high_sr_ffhq_241220_205615
  ]
  datasets:[
    train:[
      name: Train
      mode: HR
      dataroot: dataset/train_micro_64_512
      datatype: img
      l_resolution: 64
      r_resolution: 512
      batch_size: 2
      num_workers: 1
      use_shuffle: True
      data_len: -1
    ]
    val:[
      name: Validation
      mode: LRHR
      dataroot: dataset/val

##Training

Before training open sr_sr3_64_512.json file from config. Find the place that starts as:

"
train": {
        "n_iter":
"

Modify the following parameters.
* n_iter. Preloaded checkpoint has 830,000 iterations, so if, for example, you want to do 1,000 iterations, you should set it to 831,000.
* Set val_freq to how often you want to validate on validation dataset (i.e. 100 means every 100 iterations), if you don't want to validate, set it to >number of iterations.
* Same for save_checkpoint_freq, but make sure you save at least one checkpoint. So, set it to 1000 if you are doign 1000 iterations
* print_freq same, how often you want loss and updates to be printed.

##Training

Before training open sr_sr3_64_512.json file from config. Find the place that starts as:

"
train": {
        "n_iter":
"

Modify the following parameters.
* n_iter. Preloaded checkpoint has 830,000 iterations, so if, for example, you want to do 1,000 iterations, you should set it to 831,000.
* Set val_freq to how often you want to validate on validation dataset (i.e. 100 means every 100 iterations), if you don't want to validate, set it to >number of iterations.
* Same for save_checkpoint_freq, but make sure you save at least one checkpoint. So, set it to 1000 if you are doign 1000 iterations
* print_freq same, how often you want loss and updates to be printed.

In [8]:
%cd /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement

/scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement


In [31]:
!python /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/sr.py -p train -c /scratch/rv1057/Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json

export CUDA_VISIBLE_DEVICES=0,1,2
24-12-20 17:11:59.846 - INFO:   name: distributed_high_sr_ffhq
  phase: train
  gpu_ids: [0, 1, 2]
  path:[
    log: experiments/distributed_high_sr_ffhq_241220_171159/logs
    tb_logger: experiments/distributed_high_sr_ffhq_241220_171159/tb_logger
    results: experiments/distributed_high_sr_ffhq_241220_171159/results
    checkpoint: experiments/distributed_high_sr_ffhq_241220_171159/checkpoint
    resume_state: I830000_E32
    experiments_root: experiments/distributed_high_sr_ffhq_241220_171159
  ]
  datasets:[
    train:[
      name: Train
      mode: HR
      dataroot: dataset/train_FNa_64_512
      datatype: img
      l_resolution: 64
      r_resolution: 512
      batch_size: 2
      num_workers: 1
      use_shuffle: True
      data_len: -1
    ]
    val:[
      name: Validation
      mode: LRHR
      dataroot: dataset/val_FNa_64_512
      datatype: img
      l_resolution: 64
      r_resolution: 512
      data_len: 3
    ]
  ]
  model:[
    which_

Copy the resulting checkpoint (which is just weights of a model) to your drive for later use. Check the last job on experiments folder, copy its name here.

In [ ]:
%cp -r /content/Image-Super-Resolution-via-Iterative-Refinement/experiments/distributed_high_sr_ffhq_241207_015530/checkpoint /content/drive/MyDrive/checkpoints_F

Run for inference to check the results (same name as above)

In [ ]:
!python infer.py -c /content/Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json

python3: can't open file '/content/infer.py': [Errno 2] No such file or directory
